In [1]:
import os
import openai
from supabase import create_client, Client
import requests

In [2]:
supabase_url = os.environ['NEXT_PUBLIC_SUPABASE_URL']
supabase_anon_key = os.environ['NEXT_PUBLIC_SUPABASE_ANON_KEY']
openai_api_key = os.environ['OPENAI_API_KEY']

In [4]:
openai.api_key = openai_api_key
supabase: Client = create_client(supabase_url, supabase_anon_key)

In [50]:
def calculate_average_similarities(results):
    # 各URLに対する類似度の平均値を計算する。
    url_similarities = {}
    for result in results:
        url = result['url']
        similarity = result['similarity']
        if url in url_similarities:
            url_similarities[url].append(similarity)
        else:
            url_similarities[url] = [similarity]

    return {url: sum(similarities) / len(similarities) for url, similarities in url_similarities.items()}

def process_response(response):
    # Supabaseからのレスポンスを処理し、各URLの平均類似度を表示する。
    try:
        results = response.data
        average_similarities = calculate_average_similarities(results)
        for url, avg_similarity in average_similarities.items():
            print(f"URL: {url}, Average Similarity: {avg_similarity:.2f}")
    except Exception as e:
        print("エラーが発生しました: ", e)


def find_similar_documents(prompt, similarity_threshold=0.1, match_count=5):
    # プロンプトからベクトルを生成
    embedding_response = openai.Embedding.create(input=prompt, model="text-embedding-ada-002")
    query_embedding = embedding_response['data'][0]['embedding']

    # Supabase RPCを使用して類似ドキュメントを取得
    response = supabase.rpc(
        'match_documents',
        {
            'query_embedding': query_embedding,
            'similarity_threshold': similarity_threshold,
            'match_count': match_count
        }
    # 直接レスポンスオブジェクトは表示させず、レスポンスデータを取得する
    ).execute()

    process_response(response)

In [52]:
find_similar_documents("Reactについて教えてください")

2023-11-12 04:37:50,300:INFO - HTTP Request: POST https://ogvafqdckdxorurwynoi.supabase.co/rest/v1/rpc/match_documents "HTTP/1.1 200 OK"


URL: https://udemy.benesse.co.jp/development/app/react-beginner.html, Average Similarity: 0.14
URL: https://www.alobridge.com/blog/1266/, Average Similarity: 0.14
URL: https://ja.wikipedia.org/wiki/React, Average Similarity: 0.14
